In [1]:
import pandas as pd
from analysis import *

In [2]:
surge_annotations = data.surge_evaluation.annotation_dataframe()
surge_annotations_comparative = data.surge_evaluation.comparative_annotation_dataframe()

surge_annotations

0  \
bot            category        label           item                               
blender2_3B    likert dialogue proactive       (1048,38)_blender2_3B          4   
                               engaging        (1048,38)_blender2_3B          4   
                               relevant        (1048,38)_blender2_3B          3   
                               emotional       (1048,38)_blender2_3B          4   
                               grammatical     (1048,38)_blender2_3B          4   
...                                                                          ..   
rerank_blender behavior        preference info ((441,26)_rerank_blender, 14)  1   
                               life info       ((441,26)_rerank_blender, 14)  0   
                               antisocial      ((441,26)_rerank_blender, 14)  0   
                               correct fact    ((441,26)_rerank_blender, 14)  1   
                               incorrect fact  ((441,26)_rerank_blender, 14)  0   

                                                                                1  \
bot            category        label           item                                 
blender2_3B    likert dialogue proactive       (1048,38)_blender2_3B          4.0   
                               engaging        (1048,38)_blender2_3B          5.0   
                               relevant        (1048,38)_blender2_3B          4.0   
                               emotional       (1048,38)_blender2_3B          5.0   
                               grammatical     (1048,38)_blender2_3B          4.0   
...                                                                           ...   
rerank_blender behavior        preference info ((441,26)_rerank_blender, 14)  NaN   
                               life info       ((441,26)_rerank_blender, 14)  NaN   
                               antisocial      ((441,26)_rerank_blender, 14)  NaN   
                               correct fact    ((441,26)_rerank_blender, 14)  NaN   
                               incorrect fact  ((441,26)_rerank_blender, 14)  NaN   

                                                                               2  
bot            category        label           item                               
blender2_3B    likert dialogue proactive       (1048,38)_blender2_3B         NaN  
                               engaging        (1048,38)_blender2_3B         NaN  
                               relevant        (1048,38)_blender2_3B         NaN  
                               emotional       (1048,38)_blender2_3B         NaN  
                               grammatical     (1048,38)_blender2_3B         NaN  
...                                                                           ..  
rerank_blender behavior        preference info ((441,26)_rerank_blender, 14) NaN  
                               life info       ((441,26)_rerank_blender, 14) NaN  
                               antisocial      ((441,26)_rerank_blender, 14) NaN  
                               correct fact    ((441,26)_rerank_blender, 14) NaN  
                               incorrect fact  ((441,26)_rerank_blender, 14) NaN  

[151824 rows x 3 columns]

# 3 Behavior Evaluation Procedure

### Behavior Examples

In [3]:
seeds = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
for s, b in zip(seeds, behavior):
    print(get_example(
        data.surge_evaluation,
        category.behavior, b, context=0, mark=1,
        seed=s,
        annotations=surge_annotations
    ))

User:  Like he doesn't respect you or your time?
Sys:   No, he just tries to make me do things I'm not interested in. It's really annoying.
Label: antisocial = 1

User:  Yes you are right. Have you taken any trips recently
Sys:   No, I haven't, but I would love to go on a trip soon. I've always wanted to go to hawaii, but it's been a while since I've visited there.
Label: commonsense contradiction = 1

User:  I don't have a sister, but I do have a brother!
Sys:   Me neither! I have always wanted an older sister, though. Do you wish you had siblings?
Label: partner contradiction = 1

User:  speak to me in spanish
Sys:   That would be so cool! I think I'd like to learn spanish. I think it would be fun to be able to communicate with people from all over the world!
Label: self contradiction = 1

User:  i find it very confusing
Sys:   The road which leads to the road is privately owned (initially by the newds family) and payment is required to drive down the gravel road, although it can be 

### Behavior Annotation Pilot Agreements

In [4]:
@to_file
def agreement_summaries(evaluations):
    summaries = []
    for evaluation in evaluations:
        annotations = evaluation.annotation_dataframe()
        agreement = agreement_dataframe(annotations, ci=False)
        macros = agreement.dropna().mean()
        summaries.append(macros)
    sum_df = pd.concat(summaries, axis=1).transpose()
    sum_df.set_axis(
        [stat.kripp_alpha, 'x', stat.fleiss_kappa, stat.n],
        inplace=True, axis=1
    )
    sum_df.drop('x', axis=1, inplace=True)
    return sum_df

In [5]:
# todo - include ALL pilot annotations in agreement calculation (not just double annotation)
agreement_summaries(data.annotation_pilots)

,Krippendorff's alpha,Fleiss' kappa,n
0,0.112585,0.105706,65.000000
1,0.377984,0.356535,15.000000
2,0.182412,0.154556,15.500000
3,0.261712,0.172157,120.486486
4,0.351674,0.294750,41.222222


### Behavior Annotation Pilot Screening

In [6]:
across_evaluations(
    data.annotation_pilots_onboarding[2:4],
    screening_rates_by_label,
    load='results/annotation_pilot_screening'
)

attempted  passed  mistakes  accuracy   n
round                                                                
0     transitions                   4       4  6.000000  0.660621  11
      knowledge                     4       4  2.666667  0.775214  12
      consistency                   4       4  5.500000  0.647395  10
      interpretability              4       4  0.250000  0.979167   4
      commonsense                   4       4  2.000000  0.856456   8
      sociality                     4       4  0.400000  0.960000   5
1     transitions                   5       5  4.000000  0.750000   5
      knowledge                     7       6  2.833333  0.772009  12
      consistency                   4       4  5.090909  0.672116  11
      interpretability              5       5  1.000000  0.913420   7
      empathy                      10       8  4.700000  0.686667  20
      personal_information          8       7  3.500000  0.766667  16
      commonsense                   7       7  3.100000  0.777473  10
      sociality                     4       4  0.750000  0.925000   4